In [1]:
import sys
#add the path were the models are
sys.path.append("../inProduction/")
sys.path.append("../main/")
import pandas as pd
import numpy as np
from functools import reduce
from modelos import SIR, EXP, SEIR_PSO
from bootstrap import bootstrapTS as bsTS
import matplotlib.pyplot as plt
#set default theme for plts
import time

In [2]:
ci = bsTS()

In [3]:
data = pd.read_csv("../data/dadosCovid.csv")
data.head()

,regiao,estado,data,casosNovos,casosAcumulados,obitosNovos,obitosAcumulados
0,Norte,RO,2020-01-30,0,0,0,0
1,Norte,RO,2020-01-31,0,0,0,0
2,Norte,RO,2020-02-01,0,0,0,0
3,Norte,RO,2020-02-02,0,0,0,0
4,Norte,RO,2020-02-03,0,0,0,0


In [4]:
#Select only Sao Paulo
sp = data[data["estado"] == "SP"]
#Remove missing values to not crash the intervals
sp = sp.dropna()
sp.head()

,regiao,estado,data,casosNovos,casosAcumulados,obitosNovos,obitosAcumulados
1501,Sudeste,SP,2020-02-26,1,1,0,0
1502,Sudeste,SP,2020-02-27,0,1,0,0
1503,Sudeste,SP,2020-02-28,0,1,0,0
1504,Sudeste,SP,2020-02-29,1,2,0,0
1505,Sudeste,SP,2020-03-01,0,2,0,0


In [5]:
#create a series with the cummulative number of cases
y = np.array(sp["casosAcumulados"])

#Give the number of days since the day of first case confirmed
x = range(0,len(sp["casosAcumulados"]))

In [ ]:
seir = SEIR_PSO(50000000)

In [ ]:
seir.fit(x = x, y = y)

<h1>Exponencial Model</h1>

In [ ]:
exp = EXP(50000000)

In [ ]:
results = ci.single_core_CI(model = exp, 
                            y = y, 
                            x = x, 
                            ndays = 100,
                            bootstrap = 20, 
                            simulation = "Gamma_Poisson", 
                            method = "approximation",
                            model_name = "exponencial")

In [ ]:
fig, axes = plt.subplots(nrows = 1, ncols = 1, figsize = (18,12))
axes.plot(range(52,152), results[2][52:])
axes.plot(range(52,152), results[3], "--", c = "red")
axes.plot(range(52,152), results[4], "--", c = "red")

<h1>SIR with beta variante</h1>

In [ ]:
sir = SIR(50000000)

In [ ]:
results = ci.single_core_CI(model = sir, 
                             y = y, 
                             x = x, 
                             ndays = 300,
                             bootstrap = 1, 
                             simulation = "Gamma_Poisson", 
                             method = "approximation",
                             model_name = "SIR")

In [ ]:
fig, axes = plt.subplots(nrows = 1, ncols = 1, figsize = (18,12))
axes.plot(range(340,352), results[5][288:], "--", c = "red")
axes.plot(range(340,352), results[6][288:], "--", c = "red")

In [ ]:
results[4][58] - results[6][6]